In [ ]:
!pip3 install pandas matplotlib scikit-learn joblib ipywidgets

In [2]:
import pandas as pd

pd.set_option('display.max_columns', None)

structured_endometriosis_data = pd.read_csv('structured_endometriosis_data.csv')

In [ ]:
structured_endometriosis_data.isna().sum()

In [ ]:
structured_endometriosis_data.hist(bins=20, figsize=[20,20])

In [ ]:
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
model = LogisticRegression()

X = structured_endometriosis_data[['Age', 'Menstrual_Irregularity', 'Chronic_Pain_Level',
          'Hormone_Level_Abnormality', 'Infertility', 'BMI']]
y = structured_endometriosis_data['Diagnosis']


# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model.fit(X_train, y_train)

In [ ]:
predictions = model.predict(X_test)
print("Predictions:", predictions[:10])
print("Actual Labels:", y_test[:10].values)

In [ ]:
# Combine predictions and actual labels
comparison = pd.DataFrame({
    'Predicted': predictions,
    'Actual': y_test.values
})

# Display the first 10 rows
print(comparison.head(10))

In [ ]:
# Calculate accuracy
accuracy = (predictions == y_test.values).mean()
print(f"Accuracy: {accuracy:.2%}")

In [ ]:
# Add an indicator column for mismatches
comparison['Match'] = comparison['Predicted'] == comparison['Actual']

# Display rows where predictions and actual labels don't match
print(comparison[~comparison['Match']])



In [ ]:
import matplotlib.pyplot as plt

feature_names = X.columns
coefficients = model.coef_[0]

feature_importance = sorted(zip(feature_names, coefficients), key=lambda x: abs(x[1]), reverse=True)

sorted_features = [item[0] for item in feature_importance]
sorted_coefficients = [item[1] for item in feature_importance]

plt.figure(figsize=(8, 6))
plt.barh(sorted_features, sorted_coefficients, color='skyblue')
plt.axvline(0, color='gray', linewidth=0.8)
plt.title("Feature Importance (Logistic Regression)", fontsize=14)
plt.xlabel("Coefficient Value", fontsize=12)
plt.ylabel("Feature", fontsize=12)
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_predictions(y_test, predictions)

In [ ]:
import joblib

joblib.dump(model, 'endometriosis_logistic_regression_model.pkl')

In [ ]:
import ipywidgets as widgets

widgets.IntSlider()

from IPython.display import display

# # Create widgets
age_widget = widgets.IntText(min=18, description="Age:", value=18)
pain_widget = widgets.FloatSlider(min=0, max=10, description="Chronic Pain Level:", value=0)
bmi_widget = widgets.FloatSlider(min=14, max=50, description="BMI:", value=20)
menstrual_irregularity_widget = widgets.Checkbox(description="Do you experience menstrual irregularity?", value=False)
hormone_level_abnormality_widget = widgets.Checkbox(description="Do you experience hormone level abnormality?", value=False)
infertility_widget = widgets.Checkbox(description="Do you experience infertility?", value=False)

submit_button = widgets.Button(description="Submit")
result_label = widgets.Label(value="")

# # Function to handle form submission
def on_submit_button_clicked(b):
    age = age_widget.value
    pain = pain_widget.value
    bmi = bmi_widget.value
    menstrual_irregularity = menstrual_irregularity_widget.value
    hormone_level_abnormality = hormone_level_abnormality_widget.value
    infertility = infertility_widget.value
    
    
    features = pd.DataFrame([[age, menstrual_irregularity, pain, hormone_level_abnormality, infertility, bmi]],
                            columns=['Age', 'Menstrual_Irregularity', 'Chronic_Pain_Level',
                                     'Hormone_Level_Abnormality', 'Infertility', 'BMI'])
    prediction = model.predict(features)
    result_label.value = f"Prediction: {'Positive' if prediction[0] == 1 else 'Negative'}"

    
    
# # Attach function to button click event
submit_button.on_click(on_submit_button_clicked)

# # Display widgets
display(widgets.VBox([
    age_widget,
    pain_widget,
    bmi_widget,
    menstrual_irregularity_widget,
    hormone_level_abnormality_widget,
    infertility_widget,
    submit_button,
    result_label
]))